In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def make_model():

    """Beginning of create Architecture"""

    # specify the input
    inputs = tf.keras.Input(shape=(150,150,3))
    
    # specify convolutional layer 
    conv_layer = tf.keras.layers.Conv2D(filters=32,kernel_size = (3,3), activation= 'relu')

    # input goes to base model
    feature_map =conv_layer(inputs)

    # reduce the size of feature map with max pooling
    multi_vectors = tf.keras.layers.MaxPooling2D(pool_size = (2,2))(feature_map)
    
    # turn the multi-dimensional result into vectors
    vectors = tf.keras.layers.Flatten()(multi_vectors)
 
    # dense layer
    
    # dense layer with 64 neurons and relu activation
    layer1 = tf.keras.layers.Dense(64,activation='relu')(vectors)
    
    # output layer with 1 neuron and sigmoid activation
    output = tf.keras.layers.Dense(1,activation='sigmoid')(layer1)
    
    model=tf.keras.Model(inputs,output)
    
    """End of create architecture"""
   
    """
    optimizer
    """
    
    # SGD 
    optimizer = tf.keras.optimizers.SGD(learning_rate  = 0.002 , momentum = 0.8)

    loss = tf.keras.losses.BinaryCrossentropy(from_logits = False)
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

## Question 1

answer : sigmoid

## Question 2

In [3]:
model = make_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 175232)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                11214912  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
__________________________________________

## Generators and training

In [4]:
train_gen = ImageDataGenerator(
    rescale=1./255
)

train_ds = train_gen.flow_from_directory(
    './train/',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle = True
)

val_gen = ImageDataGenerator(
    rescale=1./255
)

val_ds = val_gen.flow_from_directory(
    './validation',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle = True
)



Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


## Fit the model

In [5]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 17s 173ms/step - loss: 0.7017 - accuracy: 0.5100 - val_loss: 0.6939 - val_accuracy: 0.4820
Epoch 2/10
100/100 [==============================] - 22s 223ms/step - loss: 0.6922 - accuracy: 0.5080 - val_loss: 0.6890 - val_accuracy: 0.5420
Epoch 3/10
100/100 [==============================] - 22s 223ms/step - loss: 0.6887 - accuracy: 0.5445 - val_loss: 0.6883 - val_accuracy: 0.5460
Epoch 4/10
100/100 [==============================] - 24s 242ms/step - loss: 0.6846 - accuracy: 0.5595 - val_loss: 0.6796 - val_accuracy: 0.5830
Epoch 5/10
100/100 [==============================] - 29s 294ms/step - loss: 0.6770 - accuracy: 0.5680 - val_loss: 0.6704 - val_accuracy: 0.5870
Epoch 6/10
100/100 [==============================] - 42s 421ms/step - loss: 0.6809 - accuracy: 0.5500 - val_loss: 0.6715 - val_accuracy: 0.6020
Epoch 7/10
100/100 [==============================] - 52s 520ms/step - loss: 0.6723 - accuracy: 0.5795 - val_loss: 0.6680 - val_ac

## Question 3

In [7]:
np.median( history.history['accuracy'] )

0.5637499988079071

## Question 4

In [8]:
np.std(history.history['loss'] )

0.013834974933050796

## Data augmentation

In [9]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_ds = train_gen.flow_from_directory(
    './train/',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle = True
)

val_gen = ImageDataGenerator(
    rescale=1./255
)

val_ds = val_gen.flow_from_directory(
    './validation',
    target_size = (150,150),
    batch_size = 20,
    class_mode = 'binary',
    shuffle = True
)



Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [10]:
history = model.fit(
    train_ds,
    steps_per_epoch=100,
    epochs=10,
    validation_data=val_ds,
    validation_steps=50
)

Epoch 1/10
100/100 [==============================] - 21s 214ms/step - loss: 0.6716 - accuracy: 0.5915 - val_loss: 0.6580 - val_accuracy: 0.6000
Epoch 2/10
100/100 [==============================] - 28s 285ms/step - loss: 0.6719 - accuracy: 0.5800 - val_loss: 0.6526 - val_accuracy: 0.5890
Epoch 3/10
100/100 [==============================] - 31s 312ms/step - loss: 0.6790 - accuracy: 0.5500 - val_loss: 0.6435 - val_accuracy: 0.6280
Epoch 4/10
100/100 [==============================] - 35s 346ms/step - loss: 0.6652 - accuracy: 0.5975 - val_loss: 0.6681 - val_accuracy: 0.5560
Epoch 5/10
100/100 [==============================] - 45s 448ms/step - loss: 0.6632 - accuracy: 0.5850 - val_loss: 0.6394 - val_accuracy: 0.6460
Epoch 6/10
100/100 [==============================] - 62s 615ms/step - loss: 0.6655 - accuracy: 0.5915 - val_loss: 0.6377 - val_accuracy: 0.6200
Epoch 7/10
100/100 [==============================] - 55s 555ms/step - loss: 0.6636 - accuracy: 0.5800 - val_loss: 0.6253 - val_ac

## Question 5

In [11]:
np.mean(history.history['val_loss'] )

0.64813791513443

## Question 6

In [16]:
np.mean(history.history['val_accuracy'][5:])

0.6146000027656555